In [24]:
import os
import datetime

import numpy as np
import scipy
import fiona
import statistics
import math

import pandas as pd
import geopandas as gpd
from shapely import wkt
from shapely.geometry import LineString, shape, mapping, Point, Polygon, MultiPolygon
from shapely.ops import cascaded_union

from scipy.stats import linregress

import matplotlib.pyplot as plt
from matplotlib import colors, cm, style
import matplotlib.patches as mpatches
from descartes import PolygonPatch
from matplotlib.colors import LinearSegmentedColormap

import osmnx as ox
import networkx as nx

In [25]:
from my_area_stats import val_per_area

In [26]:
style.use('seaborn-white')
input_output_folder = 'data'
osm_crs = 'EPSG:3857'

# Parameters

In [27]:
# Place                                Country              CRS
# Amsterdam | Rotterdam | The Hague    (The Netherlands)    EPSG:28992
# Madrid    | Barcelona | Valencia     (Spain)              EPSG:25830
# Stockholm | Goteborg  | Malmo        (Sweden)             EPSG:3006 ???

place_name = 'Amsterdam'
local_crs = 'EPSG:28992'
sub_folder = str(place_name).replace(' ', '')+'_14Apr2022'

In [28]:
trip_distances = [300, 500, 800]

# Load data

In [29]:
greenspaces = gpd.read_file(os.path.join(input_output_folder, sub_folder, 'greenspaces.geojson'))
greenspaces.crs = local_crs

In [30]:
paths_file = os.path.join(input_output_folder, sub_folder, 'ALTERNATIVE_betw_complete.geojson')
paths = gpd.read_file(paths_file)
paths.crs = local_crs

paths['betw_adl_800_dr12'] = paths['betw_adl_800_dr12'].astype(float)
paths['betw_adl_1000_dr12'] = paths['betw_adl_1000_dr12'].astype(float)
paths['betw_adl_1200_dr12'] = paths['betw_adl_1200_dr12'].astype(float)
paths['length'] = paths.geometry.length    

In [31]:
def set_types_geometries(df, cols):
    for col in cols:
        df[col] = df[col].apply(wkt.loads)
        gdf = gpd.GeoDataFrame(df, geometry=col)
        gdf.crs = local_crs

In [32]:
cols = []
for trip_dist in trip_distances:
    cols.append('geom_iso_'+str(trip_dist))

set_types_geometries(greenspaces, cols)

# Calculate

In [33]:
greenspaces['index'] = greenspaces.index

## Dynamic approach

In [34]:
value_cols = ['betw_adl_800_dr12', 'betw_adl_1000_dr12', 'betw_adl_1200_dr12']

### method 1: length-weighted average (i.e. people-exposure distance(~time) product)

In [35]:
greenspaces = greenspaces.set_geometry('geometry')

In [36]:
modes = ['wsum']
weight_col = 'length'

In [37]:
for value_col in value_cols:
    greenspaces = val_per_area(greenspaces, 'index', paths, modes=modes, value_col=value_col, weight_col=weight_col)

In [38]:
for value_col in value_cols:
    for mode in modes:
        col = "{}_{}".format(value_col, mode)
        greenspaces[col] = greenspaces[col].fillna(0)
        
        rank_col = "{}_{}_rank".format(value_col, mode)
        greenspaces[rank_col] = greenspaces['{}_{}'.format(value_col, mode)].rank(ascending=False, na_option='bottom', pct=True)
        greenspaces[rank_col] = greenspaces[rank_col].astype(float)

### method 2: sum entrance values / 2 (i.e. people count)

In [39]:
greenspaces['geom_boundary'] = greenspaces.geometry.boundary
greenspaces = greenspaces.set_geometry('geom_boundary')

In [40]:
entrance_paths = gpd.clip(paths, greenspaces.buffer(1)).explode().reset_index(drop=True)

/Users/roosteeuwen/opt/anaconda3/envs/greenspaces/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  """Entry point for launching an IPython kernel.


In [41]:
modes = ['sum']

In [42]:
for value_col in value_cols:
    greenspaces = val_per_area(greenspaces, 'index', entrance_paths, modes=modes, value_col=value_col)
    greenspaces[value_col+'_entering'] = greenspaces[value_col+'_sum']/2

In [43]:
for value_col in value_cols:
    for mode in modes:
        col = "{}_entering".format(value_col)
        greenspaces[col] = greenspaces[col].fillna(0)
        
        rank_col = value_col+'_entering_rank'
        greenspaces[rank_col] = greenspaces[value_col+'_entering'].rank(ascending=False, na_option='bottom', pct=True)
        greenspaces[rank_col] = greenspaces[rank_col].astype(float)

In [51]:
# fig, axs = plt.subplots(nrows=1, ncols=1, figsize=(20,20))
# fig.suptitle('Estimated commuting flows from home to adolescents\' education', fontsize=16)

# greenspaces = greenspaces.set_geometry('geometry')

# greenspaces[greenspaces[col]==0].plot(ax=axs, color='grey')
# greenspaces[greenspaces[col]>0].plot(ax=axs, column=col, cmap='Reds', scheme='quantiles')

# plt.show()

#  Store output data

In [21]:
output_sub_folder = os.path.join(input_output_folder, sub_folder)

In [22]:
if not os.path.exists(output_sub_folder):
    os.mkdir(output_sub_folder)

In [23]:
output_file = os.path.join(output_sub_folder, 'ALTERNATIVE_greenspace_accessibility.geojson')
greenspaces = greenspaces.apply(lambda c: c.astype(str) if c.name != "geometry" else c, axis=0)
greenspaces.to_file(output_file, driver='GeoJSON')